In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=448e3ced70c17691add429ac284ced59b75554d9a6597f05b492711b9bc43e0d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import wikipedia

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# Function to fetch content from Wikipedia and handle misspelled topics
def fetch_content(topic):
    try:
        content = wikipedia.page(topic).content
        return content
    except wikipedia.exceptions.DisambiguationError as e:
        print("Ambiguous topic. Did you mean one of the following?")
        for option in e.options[:5]:  # Displaying the first 5 suggestions
            print("-", option)
        return None
    except wikipedia.exceptions.PageError:
        print("Topic not found.")
        return None

# Fetch content from Wikipedia
topic = "lionel mess i"  # Example of a misspelled topic
content = fetch_content(topic)

if content:
    print("Document before preprocessing:")
    print(content[:10000])  # Print characters of the document

Document before preprocessing:
Lionel Andrés "Leo" Messi (Spanish pronunciation: [ljoˈnel anˈdɾes ˈmesi] ; born 24 June 1987) is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team. Widely regarded as one of the greatest players of all time, Messi has won a record eight Ballon d'Or awards, a record six European Golden Shoes, and was named the world's best player for a record eight times by FIFA. Until 2021, he had spent his entire professional career with Barcelona, where he won a club-record 34 trophies, including ten La Liga titles, seven Copa del Rey titles, and the UEFA Champions League four times. With his country, he won the 2021 Copa América and the 2022 FIFA World Cup. A prolific goalscorer and creative playmaker, Messi holds the records for most goals (474), hat-tricks (36), and assists in La Liga (192). He has the most international goals by a South American male (106). Messi ha

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

                    # Preprocessing
# Tokenization
tokens = word_tokenize(content)

# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tokens = [word for word in tokens if word.lower() not in stop_words]

# Stemming
porter = PorterStemmer()
stemmed_tokens = [porter.stem(word) for word in filtered_tokens]

# Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

# Remove punctuation and non-alphanumeric characters
cleaned_tokens = [re.sub(r'[^a-zA-Z0-9]', '', word) for word in lemmatized_tokens if word.isalnum()]

# Convert tokens back to a string
preprocessed_text = ' '.join(cleaned_tokens)

print("Document after preprocessing:\n")
print(preprocessed_text[:10000])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Document after preprocessing:

Lionel Andrs Leo Messi Spanish pronunciation ljonel andes mesi born 24 June 1987 Argentine professional footballer play forward captain Major League Soccer club Inter Miami Argentina national team Widely regarded one greatest player time Messi record eight Ballon award record six European Golden Shoes named world best player record eight time FIFA 2021 spent entire professional career Barcelona 34 trophy including ten La Liga title seven Copa del Rey title UEFA Champions League four time country 2021 Copa Amrica 2022 FIFA World Cup prolific goalscorer creative playmaker Messi hold record goal 474 36 assist La Liga 192 international goal South American male 106 Messi scored 800 senior career goal club country goal single club 672 Messi relocated Spain joined Barcelona aged 13 making competitive debut age 17 October 2004 established integral player club within next three year first uninterrupted season helped Barcelona achieve first treble Spanish football 

In [ ]:
!pip install gensim

In [ ]:
                                      # Load pre-trained FastText model
import gensim.downloader as api
import numpy as np
try:

    fasttext_model = api.load('fasttext-wiki-news-subwords-300')
except Exception as e:
    print("Error loading FastText model:", e)

if 'fasttext_model' in locals():
    # Get word vectors for each word in the preprocessed text
    word_vectors = []
    for word in lemmatized_tokens:
        try:
            word_vector = fasttext_model[word]
            word_vectors.append(word_vector)
        except KeyError:
            # Handle out-of-vocabulary words
            word_vectors.append(np.zeros(300))

    # Print embedding vectors #just a supset to visualize
    for i, word_vector in enumerate(word_vectors[:1000]):
        print(f"Word: {lemmatized_tokens[i]}, Embedding Vector: {word_vector}")

    # Calculate the average word vector for the entire document
    if word_vectors:
        document_vector = np.mean(word_vectors, axis=0)
       # print("Document Vector:", document_vector)
    else:
        print("No valid word vectors found.")
else:
    print("Error loading FastText model.")

In [ ]:
# Determine the character set size
char_set = set(preprocessed_text)
char_set_size = len(char_set)

# Preprocessing for character-based prediction
maxlen_char = 40  # Length of input sequence
step_char = 3     # Step size for generating input sequences
sentences_char = []  # Input sequences
next_chars = []  # Next characters

# Generate input sequences and next characters
for i in range(0, len(preprocessed_text) - maxlen_char, step_char):
    sentences_char.append(preprocessed_text[i:i + maxlen_char])
    next_chars.append(preprocessed_text[i + maxlen_char])

# Vectorization
x_char = np.zeros((len(sentences_char), maxlen_char, char_set_size), dtype=bool)
y_char = np.zeros((len(sentences_char), char_set_size), dtype=bool)

# Map characters to indices in the character set
char_to_index = {char: i for i, char in enumerate(char_set)}

for i, sentence in enumerate(sentences_char):
    for t, char in enumerate(sentence):
        x_char[i, t, char_to_index[char]] = 1
    y_char[i, char_to_index[next_chars[i]]] = 1

In [ ]:
import tensorflow as tf
# Tokenization for word-based prediction
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([preprocessed_text])
word_indices = tokenizer.word_index

# Generate input sequences and next words
sequences_word = tokenizer.texts_to_sequences([preprocessed_text])[0]
maxlen_word = 3  # Length of input sequence
step_word = 1    # Step size for generating input sequences
sentences_word = []  # Input sequences
next_words = []      # Next words

for i in range(0, len(sequences_word) - maxlen_word, step_word):
    sentences_word.append(sequences_word[i:i + maxlen_word])
    next_words.append(sequences_word[i + maxlen_word])

# Vectorization
x_word = np.array(sentences_word)
y_word = np.array(next_words)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import SimpleRNN, Dense
import numpy as np

# Define the model for character-based prediction
model_char = Sequential([
    SimpleRNN(128, input_shape=(maxlen_char, char_set_size)),
    Dense(char_set_size, activation='softmax')
])

# Compile the model
model_char.compile(loss='categorical_crossentropy', optimizer='adam')

# Train the model for character-based prediction
history_char = model_char.fit(x_char, y_char, batch_size=128, epochs=100)

# Print accuracy for character-based prediction model
print("Character-based Prediction Model:")
print("Training Loss:", history_char.history['loss'][-1])




Epoch 1/100
71/71 [==============================] - 8s 56ms/step - loss: 3.2682
Epoch 2/100
71/71 [==============================] - 3s 44ms/step - loss: 3.0123
Epoch 3/100
71/71 [==============================] - 3s 44ms/step - loss: 2.7941
Epoch 4/100
71/71 [==============================] - 2s 23ms/step - loss: 2.6235
Epoch 5/100
71/71 [==============================] - 2s 30ms/step - loss: 2.4906
Epoch 6/100
71/71 [==============================] - 2s 33ms/step - loss: 2.3871
Epoch 7/100
71/71 [==============================] - 2s 22ms/step - loss: 2.3063
Epoch 8/100
71/71 [==============================] - 2s 22ms/step - loss: 2.2402
Epoch 9/100
71/71 [==============================] - 2s 22ms/step - loss: 2.1822
Epoch 10/100
71/71 [==============================] - 2s 23ms/step - loss: 2.1277
Epoch 11/100
71/71 [==============================] - 2s 22ms/step - loss: 2.0788
Epoch 12/100
71/71 [==============================] - 2s 33ms/step - loss: 2.0323
Epoch 13/100
71/71 [=====

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
import numpy as np

# Assuming x_word and y_word are defined and preprocessed
# Check the shape of x_word
print("Shape of x_word:", x_word.shape)

# Reshape x_word to add a new dimension for time steps
x_word = np.expand_dims(x_word, axis=-1)

# Define the model for word-based prediction
model_word = Sequential([
    SimpleRNN(128, input_shape=(x_word.shape[1], x_word.shape[2])),
    Dense(len(word_indices) + 1, activation='softmax')
])

# Check the shape of x_word after reshaping
print("Shape of x_word after reshaping:", x_word.shape)

# Compile the model with accuracy as a metric
model_word.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model for word-based prediction
history_word = model_word.fit(x_word, y_word, batch_size=128, epochs=100)

# Print training loss and accuracy for word-based prediction model
print("Word-based Prediction Model:")
print("Training Loss:", history_word.history['loss'][-1])
print("Training Accuracy:", history_word.history['accuracy'][-1])


Shape of x_word: (3872, 3)
Shape of x_word after reshaping: (3872, 3, 1)
Epoch 1/100
31/31 [==============================] - 1s 5ms/step - loss: 7.2166 - accuracy: 0.0240
Epoch 2/100
31/31 [==============================] - 0s 5ms/step - loss: 6.7438 - accuracy: 0.0452
Epoch 3/100
31/31 [==============================] - 0s 5ms/step - loss: 6.6601 - accuracy: 0.0436
Epoch 4/100
31/31 [==============================] - 0s 5ms/step - loss: 6.5990 - accuracy: 0.0431
Epoch 5/100
31/31 [==============================] - 0s 5ms/step - loss: 6.5397 - accuracy: 0.0434
Epoch 6/100
31/31 [==============================] - 0s 7ms/step - loss: 6.4936 - accuracy: 0.0431
Epoch 7/100
31/31 [==============================] - 0s 8ms/step - loss: 6.4469 - accuracy: 0.0431
Epoch 8/100
31/31 [==============================] - 0s 8ms/step - loss: 6.4085 - accuracy: 0.0434
Epoch 9/100
31/31 [==============================] - 0s 7ms/step - loss: 6.3695 - accuracy: 0.0429
Epoch 10/100
31/31 [================

In [ ]:
# Define a function to generate text using the character-based SimpleRNN model for at least 10 iterations
def generate_char_text_simple(model, seed_text, char_to_index, index_to_char, char_set_size, min_iterations=10):
    generated_text = seed_text
    input_sequence = [char_to_index[char] for char in seed_text]

    # Generate text for at least 10 iterations
    for _ in range(min_iterations):
        # Prepare the input data
        x_pred = np.zeros((1, len(input_sequence), char_set_size))
        for t, char_index in enumerate(input_sequence):
            x_pred[0, t, char_index] = 1

        # Predict the next character
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = index_to_char[next_index]

        # Add the predicted character to the generated text
        generated_text += next_char
        input_sequence.append(next_index)
        input_sequence = input_sequence[1:]

    return generated_text

# Generate text starting from the initial character 'L' using character-based SimpleRNN
initial_char = 'A'
generated_char_text_simple = generate_char_text_simple(model_char, initial_char, char_to_index, index_to_char, char_set_size, min_iterations=10)
print("Generated Character-based Text :")
print(generated_char_text_simple)


Generated Character-based Text :
Ared Bald B


In [ ]:
from keras.preprocessing.sequence import pad_sequences
# Define a function to generate text using the word-based model with direct sampling
def generate_word_text_simple(model, seed_text, tokenizer, length=10):
    generated_text = seed_text
    input_sequence = tokenizer.texts_to_sequences([seed_text])[0]

    for _ in range(length):
        # Ensure the input sequence length is consistent with the model's input shape
        input_sequence_padded = pad_sequences([input_sequence], maxlen=model.input_shape[1], padding='pre')

        # Predict the next word probabilities
        preds = model.predict(input_sequence_padded, verbose=0)[0]

        # Sample the next word index based on the predicted probabilities
        next_index = np.random.choice(len(preds), p=preds)

        # Convert the next word index to the actual word
        next_word = tokenizer.index_word[next_index]

        # Add the predicted word to the generated text
        generated_text += ' ' + next_word
        input_sequence.append(next_index)
        input_sequence = input_sequence[1:]

    return generated_text

# Generate text starting from the initial word 'lionel' using direct sampling
initial_word = 'lionel'
generated_word_text = generate_word_text_simple(model_word, initial_word, tokenizer, length=10)
print("Generated Word-based Text with Direct Sampling:")
print(generated_word_text)

Generated Word-based Text with Direct Sampling:
lionel guardiola argentine nine since messi youth game contemporary 2007 time
